In [1]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()

True

In [2]:
code = os.getenv("COURSE_CODE")
learn_module = "all"

client = AzureOpenAI(azure_endpoint=os.getenv("AZURE_ENDPOINT"), api_version="2024-02-15-preview", api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def generateTranscript(mdFile, action):
    print(f"- Generating transcript ({action}) for {mdFile}")
    
    with open(mdFile, "r") as file:
        contents = file.read()
        
    with open(mdFile.replace(".md", ".questions.json"), "r", encoding="utf-8") as questions_file:
        questions = questions_file.read()
        
    with open(f"prompts/4.generatetranscript.{action}.txt", "r", encoding="utf-8") as text_file:
        systemprompt = text_file.read()
    
    systemprompt = systemprompt.replace("{contents}", contents)
    userprompt = f"""Generate the transcript, and use the following question and answers. For the answers, express the concept without relying on analogies.
                    =========
                    {questions}
                    """
                    
    message_text = [
        {"role":"system","content":systemprompt},
        {"role":"user","content":userprompt}
    ]
        
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages = message_text,
        temperature=0.1,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )    
    
    output = completion.choices[0].message.content
    print(f"- Actual total usage token={completion.usage.total_tokens}")
    
    with open(mdFile.replace(".md", ".transcript.md"), "w", encoding="utf-8") as file:
        file.write(output)

In [4]:
with open("./output/LearningPaths.json", "r") as file:
    learning_paths = json.load(file)

iModule = 1    
for lp in learning_paths:
    modules = [module for module in lp["learning_modules"] if module["learning_module"] == learn_module or learn_module == "all"]

    for module in modules:
        outputFolder_module = f"output/{code}/{iModule}.{module['learning_module']}"
        iModule += 1
        
        for index, url in enumerate(module["learning_units"]):
            unit_name = url.split("/")[-1]
            
            if index == 0:
                action = "start"
            elif index == len(module["learning_units"]) - 1:
                action = "finish"
            else:
                action = "between"
            
            outputFile_md = f"{outputFolder_module}/{unit_name}.md"
            saveLocation = f"{outputFolder_module}/{unit_name}.json"
            
            generateTranscript(outputFile_md, action)
            

- Generating transcript (start) for output/SC-900/1.Describe security and compliance concepts/1-introduction.md
- Actual total usage token=1081
- Generating transcript (between) for output/SC-900/1.Describe security and compliance concepts/2-describe-shared-responsibility-model.md
- Actual total usage token=1907
- Generating transcript (between) for output/SC-900/1.Describe security and compliance concepts/3-describe-defense-depth.md
- Actual total usage token=1887
- Generating transcript (between) for output/SC-900/1.Describe security and compliance concepts/4-describe-zero-trust-model.md
- Actual total usage token=2419
- Generating transcript (between) for output/SC-900/1.Describe security and compliance concepts/5-describe-encryption-hashing.md
- Actual total usage token=2481
- Generating transcript (finish) for output/SC-900/1.Describe security and compliance concepts/6-describe-compliance-concepts.md
- Actual total usage token=2479
- Generating transcript (start) for output/SC-900